# ———————————Recursion—————————————
## from hx

In [ ]:
def updata_mongodb(json_data,timestamp):
    if "timestamp" in json_data:
        ctime = json_data["timestamp"]
    else:
        ctime = timestamp
    update_dict = {}
    
    # tweet_type
    reply_status = False
    retweet_status = False
    quote_status = False
    try:
        if json_data["in_reply_to_status_id_str"] != None:
            reply_status = True
    except KeyError:
        pass      
    if "quoted_status" in json_data:
        quote_status = True
    if "retweeted_status" in json_data:
        retweet_status = True
    
    # format update_dict
    try_insert_dict(update_dict,"tweet_id",json_data["id_str"])
    try:
        if json_data["truncated"]==False:
            try_insert_dict(update_dict,"text",json_data["text"])
            try_insert_dict(update_dict,"hashtags",json_data["entities"]["hashtags"])
        elif json_data["truncated"]==True:
            try_insert_dict(update_dict,"text",json_data["extended_tweet"]["full_text"])
            try_insert_dict(update_dict,"hashtags",json_data["extended_tweet"]["entities"]["hashtags"])
    except KeyError:
            pass
    
    try_insert_dict(update_dict,"userid",json_data["user"]["id_str"])
    try_insert_dict(update_dict,"username",json_data["user"]["name"])
    try_insert_dict(update_dict,"user_screen_name",json_data["user"]["screen_name"])
    
    try_insert_dict(update_dict,"quote_count",json_data["quote_count"])
    try_insert_dict(update_dict,"reply_count",json_data["reply_count"])
    try_insert_dict(update_dict,"retweet_count",json_data["retweet_count"])
    try_insert_dict(update_dict,"favorite_count",json_data["favorite_count"])
    try_insert_dict(update_dict,"timestamp_ms",ctime)
    
    if reply_status == True:
        try_insert_dict(update_dict,"reply_to_tweeet_id",json_data["in_reply_to_status_id_str"])
        try_insert_dict(update_dict,"reply_to_user_id",json_data["in_reply_to_user_id_str"])
        try_insert_dict(update_dict,"reply_to_user_name",json_data["in_reply_to_screen_name"])
    if quote_status == True:
        try_insert_dict(update_dict,"quote_to_tweet_id",json_data["quoted_status_id_str"])
    if retweet_status == True:
        try_insert_dict(update_dict,"retweet_to_tweet_id",json_data["retweeted_status"]["id_str"])
    
    # reply table    
    if reply_status == True:
        exist_status = False
        doc = reply_col.find({"tweet_id":json_data["id_str"]})
        for x in doc:
            exist_status = True
        if exist_status == False:
            reply_col.insert_one(update_dict)
        else:
            reply_col.replace_one(
                {"tweet_id":json_data["id_str"],"timestamp_ms":{"$lt":ctime}},
                update_dict,
                upsert=False
                  )
    
    
    # quote table
    if quote_status == True:
        exist_status = False
        doc = quote_col.find({"tweet_id":json_data["id_str"]})
        for x in doc:
            exist_status = True
        if exist_status == False:
            quote_col.insert_one(update_dict)
        else:
            quote_col.replace_one(
                {"tweet_id":json_data["id_str"],"timestamp_ms":{"$lt":ctime}},
                update_dict,
                upsert=False
                  )      
        
        
    # retweet table
    if retweet_status == True:
        exist_status = False
        doc = retweet_col.find({"tweet_id":json_data["id_str"]})
        for x in doc:
            exist_status = True
        if exist_status == False:
            retweet_col.insert_one(update_dict)
        else:
            retweet_col.replace_one(
                {"tweet_id":json_data["id_str"],"timestamp_ms":{"$lt":ctime}},
                update_dict,
                upsert=False
                  ) 
            
            
    # origin table:
    if (reply_status==False) and (quote_status==False) and (retweet_status==False):
        exist_status = False
        doc = origin_col.find({"tweet_id":json_data["id_str"]})
        for x in doc:
            exist_status = True
        if exist_status == False:
            origin_col.insert_one(update_dict)
        else:
            origin_col.replace_one(
                {"tweet_id":json_data["id_str"],"timestamp_ms":{"$lt":ctime}},
                update_dict,
                upsert=False
                  ) 

In [ ]:
def recursive_upload(json_data,excut_function,timestamp):
    reply_status = False
    retweet_status = False
    quote_status = False
    # reply status 
    try:
        if json_data["in_reply_to_status_id_str"] != None:
            reply_status = True
    except KeyError:
        pass  
    # quote status 
    if "quoted_status" in json_data:
        quote_status = True
    # retweet status
    if "retweeted_status" in json_data:
        retweet_status = True  
    
    if (reply_status==False) and (quote_status==False) and (retweet_status==False):
        excut_function(json_data,timestamp)
    elif reply_status==True:
        excut_function(json_data,timestamp)
    elif quote_status==True:
        recursive_upload(json_data["quoted_status"],excut_function,timestamp)
        excut_function(json_data,timestamp)
    elif retweet_status==True:
        recursive_upload(json_data["retweeted_status"],excut_function,timestamp)
        excut_function(json_data,timestamp)

# ———————————start from here—————————————

In [41]:
from pyspark.sql import SparkSession, SQLContext, Row
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType, TimestampType, LongType
import json
from datetime import datetime

## Create Spark session

spark = SparkSession.builder.enableHiveSupport().config("spark.sql.legacy.createHiveTableByDefault", False).appName("UserInformation").getOrCreate()
sc = spark.sparkContext
sparksql = SQLContext(sc)

In [8]:
# Set the warehouse location
warehouse_location = "file:///home/hx152/Project/tweet_project/Dataset/spark-warehouse"
hivedb_location = "file:///home/hx152/Project/tweet_project/Dataset/metastore_db"
# warehouse_location = "hdfs://data/"

spark = SparkSession.builder \
    .enableHiveSupport() \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.warehouse.dir", warehouse_location) \
    .config("spark.sql.legacy.createHiveTableByDefault", False) \
    .appName("UserInformation").getOrCreate()
sc = spark.sparkContext
sparksql = SQLContext(sc)

23/04/23 03:48:16 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


## Create DataBase

In [9]:
a = sparksql.sql('SHOW DATABASES')
a.collect()

[Row(namespace='default'),
 Row(namespace='user_data'),
 Row(namespace='userinformation')]

In [4]:
sql_query = 'CREATE DATABASE IF NOT EXISTS userInformation'
a = sparksql.sql(sql_query)
a.collect()

[]

In [10]:
a = sparksql.sql('Use userInformation')
a.collect()

[]

## Create tables in selected DataBase

In [89]:
# DO NOT RUN. An example of data
data

{'created_at': 'Sun Apr 12 18:27:26 +0000 2020',
 'id': 1249403770360016896,
 'id_str': '1249403770360016896',
 'text': 'SERÁ APENAS COINCIDÊNCIA?\n\nApós escudo do Remo aparecer em prédio, Prefeitura divulgou que 33 pessoas com corona se… https://t.co/lCcwYH2eg5',
 'display_text_range': [0, 140],
 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
 'truncated': True,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 988174833849634816,
  'id_str': '988174833849634816',
  'name': 'TV Conservador / Metró',
  'screen_name': 'TVConservador',
  'location': 'Belém, Brasil',
  'url': None,
  'description': 'Emissora afiliada a Rede Metrópole (@cnt_pr) no estado do Pará. As notícias aqui dadas são FICTÍCIAS, não pegue corda ok 👍\n\n© Sistema O Conservador Ltda.',
  'translator_type': 'none',
  'protected': False,
  '

In [49]:
# table 'user421' add a column named 'tweet_creat_time'
sparksql.sql('''
            CREATE TABLE IF NOT EXISTS user422(
             tweet_creat_time TIMESTAMP,
             id LONG,
             name STRING,
             screen_name STRING,
             verified BOOLEAN,
             created_at TIMESTAMP,
             followers_count INT,
             friends_count INT,
             listed_count INT,
             favourites_count INT,
             statuses_count INT
            ) USING parquet
            ''')


DataFrame[]

In [92]:
sparksql.sql('''
            CREATE TABLE IF NOT EXISTS user423(
             tweet_create_time TIMESTAMP,
             tweet_id LONG,
             user_id LONG,
             name STRING,
             screen_name STRING,
             location STRING,
             url STRING,
             description STRING,
             verified BOOLEAN,
             created_at TIMESTAMP,
             followers_count INT,
             friends_count INT,
             listed_count INT,
             favourites_count INT,
             statuses_count INT 
            ) USING parquet
            ''')


DataFrame[]

In [93]:
a = sparksql.sql('SHOW TABLES')
a.collect()

[Row(namespace='userinformation', tableName='basic', isTemporary=False),
 Row(namespace='userinformation', tableName='user', isTemporary=False),
 Row(namespace='userinformation', tableName='user421', isTemporary=False),
 Row(namespace='userinformation', tableName='user422', isTemporary=False),
 Row(namespace='userinformation', tableName='user423', isTemporary=False)]

sparksql.sql('''
    DROP TABLE user423
''')


## Insert User Data from the tweet data

### ----------------------- ⭐️Schames⭐️ -----------------------

In [13]:
# define schema for user421 table
user_schema = StructType([
    StructField("tweet_creat_time", TimestampType()),
    StructField("id", LongType()),
    StructField("name", StringType()),
    StructField("screen_name", StringType()),
    StructField("verified", BooleanType()),
    StructField("created_at", TimestampType()),
    StructField("followers_count", IntegerType()),
    StructField("friends_count", IntegerType()),
    StructField("listed_count", IntegerType()),
    StructField("favourites_count", IntegerType()),
    StructField("statuses_count", IntegerType())
])

In [94]:
user423_schema = StructType([
    StructField("tweet_create_time", TimestampType()),
    StructField("tweet_id", LongType()),
    StructField("user_id", LongType()),
    StructField("name", StringType()),
    StructField("screen_name", StringType()),
    StructField("location", StringType()),
    StructField("url", StringType()),
    StructField("description", StringType()),
    StructField("verified", BooleanType()),
    StructField("created_at", TimestampType()),
    StructField("followers_count", IntegerType()),
    StructField("friends_count", IntegerType()),
    StructField("listed_count", IntegerType()),
    StructField("favourites_count", IntegerType()),
    StructField("statuses_count", IntegerType())
])

### ------------------------ ⭐️Functions⭐️ ------------------------

In [21]:
def insert_with_df(data, table_name, table_schema):
    datetime_format = "%a %b %d %H:%M:%S %z %Y"
    tweet_creat_at_datetime = datetime.strptime(data['created_at'], datetime_format)
    created_at_datetime = datetime.strptime(data['user']['created_at'], datetime_format)

    user_data_row = [(tweet_creat_at_datetime, data['user']['id'], data['user']['name'], data['user']['screen_name'], data['user']["verified"], created_at_datetime, data['user']["followers_count"], data['user']["friends_count"], data['user']["listed_count"], data['user']["favourites_count"], data['user']["statuses_count"])]

    if user_exist(data['user']['id'], table_name): # if already in the table
        update_user(user_data_row, table_name)
    else: # if does not exist in table
        insert_user(user_data_row, table_name)

def user_exist(user_id, table_name) -> bool:
    result = spark.sql(f"SELECT COUNT(*) as count FROM {table_name} WHERE id = {user_id}")
    count = result.collect()[0]["count"]
    return count > 0

def update_user(data_row, table_name):
    user_id = data_row[0][1]
    existed_ts = spark.sql(f"SELECT tweet_creat_time FROM {table_name} WHERE id = {user_id}").collect()
    if len(existed_ts) > 0:
        existed_timestamp = existed_ts[0]["tweet_creat_time"]
    else:
        existed_timestamp = None
    new_timestamp = data_row[0][0].replace(tzinfo=None)
    if existed_timestamp and new_timestamp and new_timestamp > existed_timestamp: # update only when timestamp is later
        delete_user(user_id)
        insert_user(data_row)
        
def insert_user(data_row, table_name, table_schema):
    print("！！Inserting")
    user_df = spark.createDataFrame(data_row, schema=table_schema)
    user_df.write.mode("append").insertInto(table_name)
    
def delete_user(user_id, table_name):
    spark.sql(f"DELETE FROM {table_name} WHERE id = {user_id}")


### An example for insert directly with SQL query
query = """ \
    INSERT INTO basic (id, name, screen_name) \
    VALUES (1, 'Alice', 'alice01'), \
           (2, 'Bob', 'bob02'), \
           (3, 'Charlie', 'charlie03'), \
           (4, 'David', 'david04') \
""" \
\
spark.sql(query)

Using __DataFrames__ is more efficient then inserting data directly into a table using a SQL INSERT INTO statement, especially when dealing with __large amounts of data__.

In [99]:
# CONST DEFINITION
DATETIME_FORMAT = "%a %b %d %H:%M:%S %z %Y"

In [98]:
def unify_data_type(data):
    tweet_creat_at_datetime = datetime.strptime(data['created_at'], DATETIME_FORMAT)
    created_at_datetime = datetime.strptime(data['user']['created_at'], DATETIME_FORMAT)
    user_data_row = (tweet_creat_at_datetime, data['id'], data['user']['id'], data['user']['name'], data['user']['screen_name'], data['user']['location'], data['user']['url'], data['user']['description'], data['user']["verified"], created_at_datetime, data['user']["followers_count"], data['user']["friends_count"], data['user']["listed_count"], data['user']["favourites_count"], data['user']["statuses_count"])
    return user_data_row

def insert_with_df(df, schema, table_name):    
    user_df = spark.createDataFrame(result, schema)
    user_df.write.mode("append").insertInto(table_name)

In [100]:
with open("/home/hx152/Project/tweet_project/Dataset/test_data", "r") as f1:
    
    result = []
    for line in f1:
        try:
            data = json.loads(line)
            user_data_row = unify_data_type(data)
            result.append(user_data_row)
            
            if 'retweeted_status' in data:
                data = data['retweeted_status']
                user_data_row = unify_data_type(data)
                result.append(user_data_row)
                
            if 'quoted_status' in data:
                data = data['quoted_status']
                user_data_row = unify_data_type(data)
                result.append(user_data_row)

        except json.JSONDecodeError:
            pass
    # parameters: 
    df, schema, table_name = result, user423_schema, 'user423'
    insert_with_df(df, schema, table_name)

In [101]:
spark.sql("SELECT count(*) number_rows FROM user423").show()

+-----------+
|number_rows|
+-----------+
|         13|
+-----------+



In [103]:
spark.sql("SELECT tweet_id, user_id, name, description FROM user423").show()

+-------------------+-------------------+--------------------+--------------------+
|           tweet_id|            user_id|                name|         description|
+-------------------+-------------------+--------------------+--------------------+
|1249403114614075400|1193535233242664960|          𝒎𝒆𝒍𝒆𝒌|🌍 / emigrant / m...|
|1249403770435493888|         4707764075| UpsidedownTurtle 🧢|                null|
|1249316363681910784|           14135350|    Bianna Golodryga|@CNN Senior Globa...|
|1249403770360016896| 988174833849634816|TV Conservador / ...|Emissora afiliada...|
|1249403767180668930|1242817830946508801|            juwelz v|Event Lyfe LLC .....|
|1249378751349231616|           16144221|                NUFF|instagram: @nuffs...|
|1249315454797168641|           46769281|          Gucci Mane|Major Label Recor...|
|1249326224964345857|          268218622|       Umesh Agrawal|                null|
|1249319407177744385|         1897514666|  Dharmendra Pradhan|Minister of Petro...|

In [30]:
result

[(datetime.datetime(2020, 4, 12, 18, 27, 25, tzinfo=datetime.timezone.utc),
  1242817830946508801,
  'juwelz v',
  'juwelz_v',
  False,
  datetime.datetime(2020, 3, 25, 14, 17, 28, tzinfo=datetime.timezone.utc),
  43,
  118,
  0,
  722,
  906),
 (datetime.datetime(2020, 4, 12, 18, 27, 25, tzinfo=datetime.timezone.utc),
  1225145123920588805,
  'efe09',
  'efe0927183508',
  False,
  datetime.datetime(2020, 2, 5, 19, 52, 38, tzinfo=datetime.timezone.utc),
  653,
  983,
  0,
  1255,
  4177),
 (datetime.datetime(2020, 4, 12, 18, 27, 26, tzinfo=datetime.timezone.utc),
  101007632,
  'Ravin Gupta',
  'IamRaavin',
  False,
  datetime.datetime(2010, 1, 1, 16, 24, 24, tzinfo=datetime.timezone.utc),
  499,
  537,
  2,
  4342,
  4038),
 (datetime.datetime(2020, 4, 12, 18, 27, 26, tzinfo=datetime.timezone.utc),
  1230170166614482944,
  'Carpe diem',
  'Carpedi92670638',
  False,
  datetime.datetime(2020, 2, 19, 16, 40, 9, tzinfo=datetime.timezone.utc),
  425,
  459,
  0,
  8830,
  13101),
 (dateti

In [23]:
spark.sql("SELECT id, name FROM user421").show()

+-------------------+--------------------+
|                 id|                name|
+-------------------+--------------------+
| 988174833849634816|TV Conservador / ...|
|         4707764075| UpsidedownTurtle 🧢|
|1230170166614482944|          Carpe diem|
|          101007632|         Ravin Gupta|
|1225145123920588805|               efe09|
|1242817830946508801|            juwelz v|
+-------------------+--------------------+



# ---------------------- Test code ↓ ↓ ↓------DO NOT RUN----------------------

## Insert data into tables with the required schema

In [95]:
# test function user_exist
user_exist(988174833849634816)

False

In [71]:
# TEST
def user_exist(user_id) -> bool:
    result = spark.sql(f"SELECT created_at FROM user WHERE id = {user_id}")
    rows = result.collect()

    if len(rows) > 0:
        created_at = rows[0]["created_at"]
        return created_at
    else:
        return None

In [76]:
# TEST
datetime_format = "%a %b %d %H:%M:%S %z %Y"
tweet_creat_at_datetime = datetime.strptime(data['created_at'], datetime_format)
tweet_creat_at_datetime

datetime.datetime(2020, 4, 12, 18, 27, 25, tzinfo=datetime.timezone.utc)

In [40]:
# TEST
a = spark.sql(f"SELECT tweet_creat_time FROM user421 WHERE id = 25235")
a.collect()

[]

### save data_row into dataframe(df) first

In [43]:
table_name = "user"

user_df.write \
    .mode("append") \
    .insertInto(table_name)

spark.sql("SELECT * FROM user").show()


+------------------+--------------------+-------------+--------+-------------------+---------------+-------------+------------+----------------+--------------+
|                id|                name|  screen_name|verified|         created_at|followers_count|friends_count|listed_count|favourites_count|statuses_count|
+------------------+--------------------+-------------+--------+-------------------+---------------+-------------+------------+----------------+--------------+
|988174833849634816|TV Conservador / ...|TVConservador|   false|2018-04-22 21:56:54|           7240|          566|          17|           18417|         22665|
+------------------+--------------------+-------------+--------+-------------------+---------------+-------------+------------+----------------+--------------+



## ----------------------- Write a recurence function for nested data -----------------------
### Extract user information in quoted tweet or retweet

def insert_func():
    

In [9]:
# pseudo-code
def extract_nested_info():
    if no more nested:
        store what we get so far
    else:
        extract_nested_info() # input: the inside data of the nested

SyntaxError: invalid syntax (1563028047.py, line 3)

### Created the db and tables, now we can insert data

In [21]:
import json

# try extract first user information

result = []
#insert path and replace name of the file below as needed
with open("/home/hx152/Project/tweet_project/Dataset/test_data", "r") as f1:
    for line in f1:
        result += [line]

data = json.loads(result[10])
data['user']


{'id': 988174833849634816,
 'id_str': '988174833849634816',
 'name': 'TV Conservador / Metró',
 'screen_name': 'TVConservador',
 'location': 'Belém, Brasil',
 'url': None,
 'description': 'Emissora afiliada a Rede Metrópole (@cnt_pr) no estado do Pará. As notícias aqui dadas são FICTÍCIAS, não pegue corda ok 👍\n\n© Sistema O Conservador Ltda.',
 'translator_type': 'none',
 'protected': False,
 'verified': False,
 'followers_count': 7240,
 'friends_count': 566,
 'listed_count': 17,
 'favourites_count': 18417,
 'statuses_count': 22665,
 'created_at': 'Sun Apr 22 21:56:54 +0000 2018',
 'utc_offset': None,
 'time_zone': None,
 'geo_enabled': False,
 'lang': None,
 'contributors_enabled': False,
 'is_translator': False,
 'profile_background_color': '000000',
 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png',
 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme1/bg.png',
 'profile_background_tile': False,
 'profile_link_color': '

In [7]:
# include the schema while creating tables
sparksql.sql('''
            CREATE TABLE IF NOT EXISTS basic(
             id Long,
             name STRING,
             screen_name STRING
            ) USING parquet
            ''')

DataFrame[]

In [38]:
datetime_format = "%a %b %d %H:%M:%S %z %Y"
created_at_datetime = datetime.strptime(data['user']['created_at'], datetime_format)

user_data_row = [(data['user']['id'], data['user']['name'], data['user']['screen_name'], data['user']["verified"], created_at_datetime, data['user']["followers_count"], data['user']["friends_count"], data['user']["listed_count"], data['user']["favourites_count"], data['user']["statuses_count"])]

user_df = spark.createDataFrame(user_data_row, schema=user_schema)

In [28]:
basic_schema = StructType([
    StructField("id", LongType()),
    StructField("name", StringType()),
    StructField("screen_name", StringType())
    ])

data_row = [(data['user']['id'], data['user']['name'], data['user']['screen_name'])]

df = spark.createDataFrame(data_row, schema=basic_schema)

In [33]:
id = data['user']['id']
name = data['user']['name']
screen_name = data['user']['screen_name']

In [60]:
one_data_row = [(id, name, screen_name)]
df = spark.createDataFrame(one_data_row, schema=basic_schema)
df.collect()

[Row(id=804046791348015107, name='Bi Sex Uau', screen_name='B_King69')]

In [30]:
# an example of insert from df
table_name = "basic"

df.write \
    .mode("append") \
    .insertInto(table_name)

# check what records have been included in basic table
spark.sql("SELECT * FROM basic").show()


+------------------+--------------------+-------------+
|                id|                name|  screen_name|
+------------------+--------------------+-------------+
|988174833849634816|TV Conservador / ...|TVConservador|
+------------------+--------------------+-------------+



In [31]:
# another way to check what records have been included in basic table
query = '''
    SELECT *
    FROM basic
'''
a = sparksql.sql(query)
a.collect()

[Row(id=988174833849634816, name='TV Conservador / Metró', screen_name='TVConservador')]

### --------------------------------------- other test code ---------------------------------------

In [ ]:
q = ''
a = sparksql.sql(q)
a.collect()

In [17]:
# define schema for user table
user_schema = StructType([
    StructField("id", IntegerType()),
    StructField("id_str", StringType()),
    StructField("name", StringType()),
    StructField("screen_name", StringType()),
    StructField("location", StringType()),
    StructField("url", StringType()),
    StructField("description", StringType()),
    StructField("translator_type", StringType()),
    StructField("protected", BooleanType()),
    StructField("verified", BooleanType()),
    StructField("created_at", TimestampType()),
    StructField("utc_offset", StringType()),
    StructField("time_zone", StringType()),
    StructField("geo_enabled", BooleanType()),
    StructField("lang", StringType()),
    StructField("contributors_enabled", BooleanType()),
    StructField("is_translator", BooleanType())
])

In [18]:
# define schema for user_profile table
profile_schema = StructType([
    StructField("id", IntegerType()),
    StructField("followers_count", IntegerType()),
    StructField("friends_count", IntegerType()),
    StructField("listed_count", IntegerType()),
    StructField("favourites_count", IntegerType()),
    StructField("statuses_count", IntegerType()),
    StructField("profile_background_color", StringType()),
    StructField("profile_background_image_url", StringType()),
    StructField("profile_background_image_url_https", StringType()),
    StructField("profile_background_tile", BooleanType()),
    StructField("profile_link_color", StringType()),
    StructField("profile_sidebar_border_color", StringType()),
    StructField("profile_sidebar_fill_color", StringType()),
    StructField("profile_text_color", StringType()),
    StructField("profile_use_background_image", BooleanType()),
    StructField("profile_image_url", StringType()),
    StructField("profile_image_url_https", StringType()),
    StructField("profile_banner_url", StringType()),
    StructField("default_profile", BooleanType()),
    StructField("default_profile_image", BooleanType())
])

In [8]:
result = []
print(len(result))
#insert path and replace name of the file below as needed
with open("/home/hx152/Project/tweet_project/Dataset/corona-out-3", "r") as f1:
    for line in f1:
        result += [line]

for i in range(num_record):
    try:
        data = json.loads(result[i])
        result += data['user']
    except json.JSONDecodeError:
        pass


0
4153117


In [48]:
username =  "x"

In [50]:
print("insert username " + username)

insert username x
